In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# --- настройки путей ---
DATA_DIR = Path(".")

In [3]:
# --- 1. загрузка сырых данных ---
data = pd.read_csv(DATA_DIR / "data.csv", sep=";", encoding="cp1251")
data2 = pd.read_csv(DATA_DIR / "data2.csv", sep=";", encoding="cp1251")

In [4]:
print("data.shape:", data.shape)
print("data2.shape:", data2.shape)

data.shape: (13114, 7)
data2.shape: (8588, 19)


In [5]:
# --- 2. переименование колонок в data (транзакции) ---

# вариант 1: уже английские названия (как в примере cst_dim_id;transdate;...)
if "cst_dim_id" in data.columns:
    data = data.rename(columns={
        "cst_dim_id": "client_id",
        "transdate": "transdate",
        "transdatetime": "transdatetime",
        "amount": "amount",
        "docno": "transaction_id",
        "direction": "destination_id",   # тут direction/destination_id – по датасету
        "target": "is_fraud",
    })
# вариант 2: русские названия (из описания хакатона)
elif "Уникальный идентификатор клиента" in data.columns:
    data = data.rename(columns={
        "Уникальный идентификатор клиента": "client_id",
        "Дата совершенной транзакции": "transdate",
        "Дата и время совершенной транзакции": "transdatetime",
        "Сумма совершенного перевода": "amount",
        "Уникальный идентификатор транзакции": "transaction_id",
        "Зашифрованный идентификатор получателя/destination транзакции": "destination_id",
        "Размеченные транзакции(переводы), где 1 - мошенническая операция , 0 - чистая": "is_fraud",
    })

# --- 3. переименование колонок в data2 (поведенческие фичи) ---

if "cst_dim_id" in data2.columns:
    data2 = data2.rename(columns={
        "cst_dim_id": "client_id",
        "transdate": "transdate",
    })
elif "Уникальный идентификатор клиента" in data2.columns:
    data2 = data2.rename(columns={
        "Уникальный идентификатор клиента": "client_id",
        "Дата совершенной транзакции": "transdate",
    })




In [6]:
# --- 4. парсинг дат с учётом формата '2025-01-05 00:00:00.000' ---
def parse_datetime_column(series: pd.Series) -> pd.Series:
    """
    Приводим строки вида '2025-01-05 00:00:00.000' к datetime.
    Убираем лишние одинарные кавычки.
    """
    return pd.to_datetime(
        series.astype(str).str.strip().str.strip("'"),
        format="%Y-%m-%d %H:%M:%S.%f",
        errors="coerce"
    )


In [7]:
# transdate и transdatetime в data
data["transdate"] = parse_datetime_column(data["transdate"])
data["transdatetime"] = parse_datetime_column(data["transdatetime"])

# transdate в data2
data2["transdate"] = parse_datetime_column(data2["transdate"])

# выбрасываем строки, где дата не распарсилась (если таких мало)
data = data.dropna(subset=["client_id", "transdate", "transdatetime"]).copy()
data2 = data2.dropna(subset=["client_id", "transdate"]).copy()

# переводим в "чистую дату" для ключа мерджа
data["trans_date"] = data["transdate"].dt.date
data2["trans_date"] = data2["transdate"].dt.date

# целевую метку в int
if data["is_fraud"].dtype != "int64":
    data["is_fraud"] = data["is_fraud"].astype(int)

print("\nПосле очистки:")
print("data.shape:", data.shape)
print("data2.shape:", data2.shape)


После очистки:
data.shape: (13107, 8)
data2.shape: (8579, 20)


In [8]:
# --- 5. мердж транзакций и поведенческих фичей ---

# чтобы не дублировать колонку transdate при merge
data2_for_merge = data2.drop(columns=["transdate"])

df = data.merge(
    data2_for_merge,
    on=["client_id", "trans_date"],
    how="left"
)

print("\nИтоговый df.shape:", df.shape)
print("\nПример строк:")
display(df.head())



Итоговый df.shape: (13126, 25)

Пример строк:


,client_id,transdate,transdatetime,amount,transaction_id,destination_id,is_fraud,trans_date,Количество разных версий ОС (os_ver) за последние 30 дней до transdate — сколько разных ОС/версий использовал клиент,Количество разных моделей телефона (phone_model) за последние 30 дней — насколько часто клиент “менял устройство” по логам,...,Среднее число логинов в день за последние 30 дней: logins_last_30_days / 30,"Относительное изменение частоты логинов за 7 дней к средней частоте за 30 дней:\n(freq7d?freq30d)/freq30d(freq_{7d} - freq_{30d}) / freq_{30d}(freq7d?freq30d)/freq30d — показывает, стал клиент заходить чаще или реже недавно",Доля логинов за 7 дней от логинов за 30 дней,Средний интервал (в секундах) между соседними сессиями за последние 30 дней,"Стандартное отклонение интервалов между логинами за 30 дней (в секундах), измеряет разброс интервалов","Дисперсия интервалов между логинами за 30 дней (в секундах?), ещё одна мера разброса","Экспоненциально взвешенное среднее интервалов между логинами за 7 дней, где более свежие сессии имеют больший вес (коэффициент затухания 0.3)",Показатель “взрывности” логинов: (std?mean)/(std+mean)(std - mean)/(std + mean)(std?mean)/(std+mean) для интервалов,Fano-factor интервалов: variance / mean,"Z-скор среднего интервала за последние 7 дней относительно среднего за 30 дней: насколько сильно недавние интервалы отличаются от типичных, в единицах стандартного отклонения"
0,2937833270,2025-01-05,2025-01-05 16:32:02,31000.0,5343,8406e407421ec28bd5f445793ef64fd1,0,2025-01-05,1,1,...,1.5333333333333334,0.21118012422360244,0.2826086956521739,49814.117647058825,106759.60669047953,11397613620.705883,18227.846188802367,0.3636976081673756,228802.8807708658,-0.21313414644761858
1,2096229005,2025-03-04,2025-03-04 17:41:57,4000.0,8442,b3a3d4a6006293195d998957d4f01e42,0,2025-03-04,1,1,...,1.7666666666666666,-0.11051212938005389,0.20754716981132076,50667.857142857145,78912.7269079706,6227218468.051948,6872.638130014896,0.21797146518520444,122902.73990657259,0.026790898502448222
2,2937759666,2025-06-20,2025-06-20 10:08:07,3000.0,9540,22b84292f0ebce65ad0808342615a03b,0,2025-06-20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2933493153,2025-07-06,2025-07-06 14:52:13,500.0,11685,d677d4e1a0f625e1ad746ea950c9dca9,0,2025-07-06,0,0,...,0.0,NaN,NaN,1350.0,1824.3354954612926,3328200.0,-1.0,0.14942828070300188,2465.3333333333335,-1.0
4,456000634,2024-12-18,2024-12-18 14:12:25,20000.0,7128,87b698d1edae13c21ce86678de3b8546,0,2024-12-18,1,1,...,0.43333333333333335,1.3076923076923077,0.5384615384615384,213360.0,425545.9700196913,"1,81E+11",127828.03317618543,0.33210829132360686,848750.3402699663,-0.4085810047547966


In [9]:
print("\nПропуски по столбцам:")
display(df.isna().sum())


Пропуски по столбцам:


client_id                                                                                                                                                                                                                            0
transdate                                                                                                                                                                                                                            0
transdatetime                                                                                                                                                                                                                        0
amount                                                                                                                                                                                                                               0
transaction_id                                                              

In [10]:
df_model = df.dropna(subset=[
    'Количество разных версий ОС (os_ver) за последние 30 дней до transdate — сколько разных ОС/версий использовал клиент',
    'Количество разных моделей телефона (phone_model) за последние 30 дней — насколько часто клиент “менял устройство” по логам',
    # ... остальные фичи из data2
]).copy()


In [11]:
df = df_model.copy()

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb
import joblib

In [13]:
# ---------------------------------------------------------------------
# 1. КОПИЯ ОБЪЕДИНЁННОГО ДАТАФРЕЙМА
# ---------------------------------------------------------------------
# предполагаем, что у тебя уже есть df после merge data + data2
df_proc = df.copy()

In [14]:
# ---------------------------------------------------------------------
# 2. РЕЙНЕЙМ РУССКИХ КОЛОНОК В EN SNAKE_CASE
#    (по startswith, чтобы не мучиться с длинными строками)
# ---------------------------------------------------------------------
rename_map = {}

for col in df_proc.columns:
    if col.startswith('Количество разных версий ОС'):
        rename_map[col] = 'os_ver_cnt_30d'
    elif col.startswith('Количество разных моделей телефона'):
        rename_map[col] = 'phone_model_cnt_30d'
    elif col.startswith('Модель телефона из самой последней сессии'):
        rename_map[col] = 'phone_model_last'
    elif col.startswith('Версия ОС из самой последней сессии'):
        rename_map[col] = 'os_version_last'
    elif col.startswith('Количество уникальных логин-сессий (минутных тайм-слотов) за последние 7 дней'):
        rename_map[col] = 'login_sessions_7d'
    elif col.startswith('Количество уникальных логин-сессий за последние 30 дней'):
        rename_map[col] = 'login_sessions_30d'
    elif col.startswith('Среднее число логинов в день за последние 7 дней'):
        rename_map[col] = 'logins_per_day_7d'
    elif col.startswith('Среднее число логинов в день за последние 30 дней'):
        rename_map[col] = 'logins_per_day_30d'
    elif col.startswith('Относительное изменение частоты логинов'):
        rename_map[col] = 'login_freq_change_7d_vs_30d'
    elif col.startswith('Доля логинов за 7 дней'):
        rename_map[col] = 'logins_7d_share_of_30d'
    elif col.startswith('Средний интервал (в секундах) между соседними сессиями за последние 30 дней'):
        rename_map[col] = 'avg_session_interval_30d'
    elif col.startswith('Стандартное отклонение интервалов между логинами за 30 дней'):
        rename_map[col] = 'std_session_interval_30d'
    elif col.startswith('Дисперсия интервалов между логинами за 30 дней'):
        rename_map[col] = 'var_session_interval_30d'
    elif col.startswith('Экспоненциально взвешенное среднее интервалов между логинами за 7 дней'):
        rename_map[col] = 'ewm_session_interval_7d'
    elif col.startswith('Показатель “взрывности” логинов'):
        rename_map[col] = 'burstiness_sessions'
    elif col.startswith('Fano-factor'):
        rename_map[col] = 'fano_factor_sessions'
    elif col.startswith('Z-скор среднего интервала'):
        rename_map[col] = 'zscore_interval_7d_vs_30d'

df_proc = df_proc.rename(columns=rename_map)

# проверим, что всё ок
print("Колонки после rename:")
print(df_proc.columns.tolist())

Колонки после rename:
['client_id', 'transdate', 'transdatetime', 'amount', 'transaction_id', 'destination_id', 'is_fraud', 'trans_date', 'os_ver_cnt_30d', 'phone_model_cnt_30d', 'phone_model_last', 'os_version_last', 'login_sessions_7d', 'login_sessions_30d', 'logins_per_day_7d', 'logins_per_day_30d', 'login_freq_change_7d_vs_30d', 'logins_7d_share_of_30d', 'avg_session_interval_30d', 'std_session_interval_30d', 'var_session_interval_30d', 'ewm_session_interval_7d', 'burstiness_sessions', 'fano_factor_sessions', 'zscore_interval_7d_vs_30d']


In [15]:
# ---------------------------------------------------------------------
# 3. ОБРАБОТКА ПРОПУСКОВ И ТИПОВ
# ---------------------------------------------------------------------

# 3.1. Категориальные фичи, которые надо закодировать
cat_cols = ['phone_model_last', 'os_version_last']
cat_cols = [c for c in cat_cols if c in df_proc.columns]  # на всякий случай

encoders = {}
for col in cat_cols:
    df_proc[col] = df_proc[col].fillna('unknown').astype(str)
    le = LabelEncoder()
    df_proc[col] = le.fit_transform(df_proc[col])
    encoders[col] = le  # можно сохранить потом для инференса

# 3.2. Числовые фичи из поведенческих + amount
num_cols = [
    'amount',
    'os_ver_cnt_30d',
    'phone_model_cnt_30d',
    'login_sessions_7d',
    'login_sessions_30d',
    'logins_per_day_7d',
    'logins_per_day_30d',
    'login_freq_change_7d_vs_30d',
    'logins_7d_share_of_30d',
    'avg_session_interval_30d',
    'std_session_interval_30d',
    'var_session_interval_30d',
    'ewm_session_interval_7d',
    'burstiness_sessions',
    'fano_factor_sessions',
    'zscore_interval_7d_vs_30d',
]

# фильтруем на случай, если чего-то нет
num_cols = [c for c in num_cols if c in df_proc.columns]

for col in num_cols:
    # приводим к виду, который можно конвертить в float
    df_proc[col] = (
        df_proc[col]
        .astype(str)
        .str.replace(',', '.', regex=False)  # вдруг запятая вместо точки
        .str.replace(' ', '', regex=False)   # убираем пробелы
    )
    df_proc[col] = pd.to_numeric(df_proc[col], errors='coerce')

    # добавим флаг пропуска
    df_proc[col + '_was_missing'] = df_proc[col].isna().astype(int)

    # заполним NaN нулями (или можно median, если захочешь)
    df_proc[col] = df_proc[col].fillna(0.0)

# убедимся, что среди фич не осталось object
print("\nТипы df_proc (object):")
print(df_proc.dtypes[df_proc.dtypes == 'object'])



Типы df_proc (object):
client_id         object
transaction_id    object
destination_id    object
trans_date        object
dtype: object


In [16]:
# ---------------------------------------------------------------------
# 4. ФОРМИРУЕМ X, y ДЛЯ ОБУЧЕНИЯ XGBoost
# ---------------------------------------------------------------------

target_col = 'is_fraud'

# служебные колонки, которые НЕ должны идти в модель
drop_cols = [
    target_col,
    'client_id',
    'transaction_id',
    'destination_id',
    'transdate',
    'transdatetime',
    'trans_date',
]

drop_cols = [c for c in drop_cols if c in df_proc.columns]

y = df_proc[target_col].astype(int)
X = df_proc.drop(columns=drop_cols)

# финальная проверка: никаких object в X быть не должно
print("\nПроверка типов в X (object-колонки, если есть):")
print(X.dtypes[X.dtypes == 'object'])


Проверка типов в X (object-колонки, если есть):
Series([], dtype: object)


In [17]:
# ---------------------------------------------------------------------
# 5. TRAIN/VALID SPLIT + ОБУЧЕНИЕ XGBoost
# ---------------------------------------------------------------------
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos
print("\nscale_pos_weight:", scale_pos_weight)

model = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    tree_method='hist',
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1,
    random_state=42,
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=50
)

y_proba = model.predict_proba(X_valid)[:, 1]
roc = roc_auc_score(y_valid, y_proba)
pr_auc = average_precision_score(y_valid, y_proba)

print(f"\nROC-AUC: {roc:.4f}")
print(f"PR-AUC:  {pr_auc:.4f}")


scale_pos_weight: 82.59016393442623
[0]	validation_0-auc:0.65849
[50]	validation_0-auc:0.85534
[100]	validation_0-auc:0.86250
[150]	validation_0-auc:0.88176
[200]	validation_0-auc:0.88206
[250]	validation_0-auc:0.88299
[300]	validation_0-auc:0.87958
[350]	validation_0-auc:0.87873
[399]	validation_0-auc:0.87983

ROC-AUC: 0.8798
PR-AUC:  0.4767


In [18]:
# ---------------------------------------------------------------------
# 6. СОХРАНЕНИЕ МОДЕЛИ
# ---------------------------------------------------------------------
joblib.dump(model, "model_xgb_baseline.pkl")
print("\nМодель сохранена в temp/model_xgb_baseline.pkl")


Модель сохранена в temp/model_xgb_baseline.pkl


In [19]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score, average_precision_score
import numpy as np
import pandas as pd

# Считаем вероятности на валидации
y_proba_valid = model.predict_proba(X_valid)[:, 1]

# Сетка порогов
thresholds = np.linspace(0.0, 1.0, 21)

rows = []
for thr in thresholds:
    y_pred = (y_proba_valid >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_valid, y_pred).ravel()

    precision = precision_score(y_valid, y_pred, zero_division=0)
    recall = recall_score(y_valid, y_pred, zero_division=0)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    tpr = recall

    rows.append({
        "threshold": thr,
        "precision": precision,
        "recall": recall,
        "FPR": fpr,
        "TPR": tpr,
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "TN": tn,
    })

thr_df = pd.DataFrame(rows)
display(thr_df)

print("Global ROC-AUC:", roc_auc_score(y_valid, y_proba_valid))
print("Global PR-AUC: ", average_precision_score(y_valid, y_proba_valid))


,threshold,precision,recall,FPR,TPR,TP,FP,FN,TN
0,0.00,0.011765,1.000000,1.000000,1.000000,30,2520,0,0
1,0.05,0.105882,0.600000,0.060317,0.600000,18,152,12,2368
2,0.10,0.153061,0.500000,0.032937,0.500000,15,83,15,2437
3,0.15,0.238095,0.500000,0.019048,0.500000,15,48,15,2472
4,0.20,0.306122,0.500000,0.013492,0.500000,15,34,15,2486
5,0.25,0.348837,0.500000,0.011111,0.500000,15,28,15,2492
6,0.30,0.400000,0.466667,0.008333,0.466667,14,21,16,2499
7,0.35,0.464286,0.433333,0.005952,0.433333,13,15,17,2505
8,0.40,0.461538,0.400000,0.005556,0.400000,12,14,18,2506
9,0.45,0.545455,0.400000,0.003968,0.400000,12,10,18,2510


Global ROC-AUC: 0.8798280423280423
Global PR-AUC:  0.4766571642491799


In [20]:
importances = model.feature_importances_
feat_importance = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importances,
}).sort_values("importance", ascending=False)

top_10_importance = feat_importance.head(10)
display(top_10_importance)


,feature,importance
0,amount,0.145602
4,os_version_last,0.114014
13,var_session_interval_30d,0.068099
2,phone_model_cnt_30d,0.059718
3,phone_model_last,0.055741
6,login_sessions_30d,0.055010
10,logins_7d_share_of_30d,0.052085
9,login_freq_change_7d_vs_30d,0.046733
17,zscore_interval_7d_vs_30d,0.046364
15,burstiness_sessions,0.045432


In [21]:
top_10_importance.to_csv("top10_feature_importance.csv", index=False)

In [22]:
# Средние и std по train-фичам
train_means = X_train.mean()
train_stds = X_train.std(ddof=0).replace(0, 1e-9)  # чтобы не делить на 0

# Соединяем с важностью
feat_stats = pd.DataFrame({
    "feature": X_train.columns,
    "mean": train_means.values,
    "std": train_stds.values,
    "importance": model.feature_importances_,
}).set_index("feature")


Функция объяснения одной транзакции

In [23]:
def explain_transaction(
    x_row: pd.Series,
    feat_stats: pd.DataFrame,
    z_threshold: float = 2.0,
    top_k: int = 10,
) -> pd.DataFrame:
    """
    x_row: одна строка X (например, X_valid.iloc[0])
    Возвращает таблицу: feature, value, mean, std, z_score, importance, is_outlier
    """
    x_row = x_row.copy()
    df = feat_stats.copy()

    df["value"] = x_row[df.index]
    df["z_score"] = (df["value"] - df["mean"]) / df["std"]
    df["abs_z"] = df["z_score"].abs()
    df["is_outlier"] = df["abs_z"] >= z_threshold

    # Сортируем по комбинации важности и отклонения
    df["score"] = df["importance"] * df["abs_z"]
    df_sorted = df.sort_values("score", ascending=False)

    return df_sorted[["value", "mean", "std", "z_score", "importance", "is_outlier"]].head(top_k)


Пример использования

In [24]:
# Берём какую-нибудь транзакцию из валидации
idx = 0
x_example = X_valid.iloc[idx]
y_true = y_valid.iloc[idx]
y_prob = y_proba_valid[idx]

print("True label:", y_true, "Predicted proba:", y_prob)

local_expl = explain_transaction(x_example, feat_stats, z_threshold=2.0, top_k=10)
display(local_expl)


True label: 0 Predicted proba: 6.265007e-05


,value,mean,std,z_score,importance,is_outlier
feature,,,,,,
os_version_last,4.100000e+01,1.761355e+01,1.732264e+01,1.350051,0.114014,False
phone_model_cnt_30d,2.000000e+00,1.315356e+00,5.546975e-01,1.234266,0.059718,False
amount,1.650000e+03,4.715892e+04,1.383952e+05,-0.328833,0.145602,False
phone_model_last,2.440000e+02,1.804137e+02,7.592444e+01,0.837494,0.055741,False
os_ver_cnt_30d,2.000000e+00,1.368111e+00,6.552180e-01,0.964394,0.027885,False
std_session_interval_30d,7.149274e+04,1.424914e+05,1.417161e+05,-0.500992,0.040760,False
var_session_interval_30d,5.111211e+09,4.038773e+10,1.293923e+11,-0.272632,0.068099,False
fano_factor_sessions,1.166994e+05,2.395168e+05,2.798793e+05,-0.438823,0.041629,False
avg_session_interval_30d,4.379810e+04,1.018146e+05,1.409876e+05,-0.411501,0.044387,False


In [25]:
# Убедимся, что transdatetime — datetime64
print(df_proc["transdatetime"].dtype)

# сортировка по времени
df_sorted = df_proc.sort_values("transdatetime").reset_index(drop=True)

# тот же список drop_cols, который ты уже использовал
drop_cols = [
    "is_fraud",
    "client_id",
    "transaction_id",
    "destination_id",
    "transdate",
    "transdatetime",
    "trans_date",
]
drop_cols = [c for c in drop_cols if c in df_sorted.columns]

X_time = df_sorted.drop(columns=drop_cols)
y_time = df_sorted["is_fraud"].astype(int)

split_idx = int(len(df_sorted) * 0.8)
X_train_time = X_time.iloc[:split_idx]
y_train_time = y_time.iloc[:split_idx]
X_test_time = X_time.iloc[split_idx:]
y_test_time = y_time.iloc[split_idx:]

X_train_time.shape, X_test_time.shape


datetime64[ns]


((10198, 34), (2550, 34))

In [26]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, average_precision_score

neg, pos = np.bincount(y_train_time)
scale_pos_weight_time = neg / pos
print("scale_pos_weight (time-based):", scale_pos_weight_time)

model_time = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    tree_method='hist',
    scale_pos_weight=scale_pos_weight_time,
    n_jobs=-1,
    random_state=42,
)

model_time.fit(
    X_train_time, y_train_time,
    eval_set=[(X_test_time, y_test_time)],
    verbose=50
)

y_proba_time = model_time.predict_proba(X_test_time)[:, 1]
roc_time = roc_auc_score(y_test_time, y_proba_time)
pr_time = average_precision_score(y_test_time, y_proba_time)

print(f"Time-based ROC-AUC: {roc_time:.4f}")
print(f"Time-based PR-AUC:  {pr_time:.4f}")


scale_pos_weight (time-based): 253.95
[0]	validation_0-auc:0.49201
[50]	validation_0-auc:0.70297
[100]	validation_0-auc:0.71674
[150]	validation_0-auc:0.71442
[200]	validation_0-auc:0.71583
[250]	validation_0-auc:0.71492
[300]	validation_0-auc:0.71607
[350]	validation_0-auc:0.71985
[399]	validation_0-auc:0.71664
Time-based ROC-AUC: 0.7166
Time-based PR-AUC:  0.1189


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
import time

results = []

def evaluate_model(name, clf, X_tr, y_tr, X_te, y_te):
    t0 = time.perf_counter()
    clf.fit(X_tr, y_tr)
    fit_time = time.perf_counter() - t0

    t1 = time.perf_counter()
    y_proba = clf.predict_proba(X_te)[:, 1]
    infer_time = time.perf_counter() - t1

    roc = roc_auc_score(y_te, y_proba)
    pr = average_precision_score(y_te, y_proba)
    time_per_1000 = infer_time * 1000 / len(X_te)

    results.append({
        "model": name,
        "ROC_AUC": roc,
        "PR_AUC": pr,
        "fit_time_sec": fit_time,
        "infer_time_per_1000_samples_ms": time_per_1000,
    })

# 1) Logistic Regression (с балансом классов)
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1,
    solver="saga",  # или "liblinear" если данных не слишком много
)

evaluate_model("LogisticRegression", log_reg, X_train, y_train, X_valid, y_valid)

# 2) RandomForest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=7,
    n_jobs=-1,
    class_weight="balanced_subsample",
    random_state=42,
)

evaluate_model("RandomForest", rf, X_train, y_train, X_valid, y_valid)

# 3) XGBoost (твоя модель)
t0 = time.perf_counter()
y_proba_xgb = model.predict_proba(X_valid)[:, 1]
infer_time_xgb = time.perf_counter() - t0

roc_xgb = roc_auc_score(y_valid, y_proba_xgb)
pr_xgb = average_precision_score(y_valid, y_proba_xgb)
time_per_1000_xgb = infer_time_xgb * 1000 / len(X_valid)

results.append({
    "model": "XGBoost",
    "ROC_AUC": roc_xgb,
    "PR_AUC": pr_xgb,
    "fit_time_sec": None,  # уже обучен
    "infer_time_per_1000_samples_ms": time_per_1000_xgb,
})

baseline_df = pd.DataFrame(results)
display(baseline_df)


,model,ROC_AUC,PR_AUC,fit_time_sec,infer_time_per_1000_samples_ms
0,LogisticRegression,0.373267,0.009415,1.413712,0.000835
1,RandomForest,0.847381,0.359697,0.484645,0.005373
2,XGBoost,0.879828,0.476657,NaN,0.001232


In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score, average_precision_score

# 1. Получаем вероятности на валидации
y_proba_valid = model.predict_proba(X_valid)[:, 1]

# 2. Строим таблицу по порогам
thresholds = np.linspace(0.0, 1.0, 101)  # шаг 0.01

rows = []
for thr in thresholds:
    y_pred = (y_proba_valid >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_valid, y_pred).ravel()

    precision = precision_score(y_valid, y_pred, zero_division=0)
    recall = recall_score(y_valid, y_pred, zero_division=0)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0

    rows.append({
        "threshold": thr,
        "precision": precision,
        "recall": recall,
        "FPR": fpr,
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "TN": tn,
    })

thr_df = pd.DataFrame(rows)
display(thr_df.head())

print("Global ROC-AUC:", roc_auc_score(y_valid, y_proba_valid))
print("Global PR-AUC: ", average_precision_score(y_valid, y_proba_valid))


,threshold,precision,recall,FPR,TP,FP,FN,TN
0,0.00,0.011765,1.000000,1.000000,30,2520,0,0
1,0.01,0.053215,0.800000,0.169444,24,427,6,2093
2,0.02,0.066456,0.700000,0.117063,21,295,9,2225
3,0.03,0.080169,0.633333,0.086508,19,218,11,2302
4,0.04,0.092784,0.600000,0.069841,18,176,12,2344


Global ROC-AUC: 0.8798280423280423
Global PR-AUC:  0.4766571642491799


In [29]:
# Правила выбора (можно подстроить):
# low: FPR <= 0.005 (очень мало ложных срабатываний)
# medium: FPR <= 0.02 и recall >= 0.5
# high: recall >= 0.8 и FPR <= 0.05

low_candidates = thr_df[thr_df["FPR"] <= 0.005]
medium_candidates = thr_df[(thr_df["FPR"] <= 0.02) & (thr_df["recall"] >= 0.5)]
high_candidates = thr_df[(thr_df["FPR"] <= 0.05) & (thr_df["recall"] >= 0.8)]

low_thr = low_candidates["threshold"].max() if not low_candidates.empty else 0.5
med_thr = medium_candidates["threshold"].max() if not medium_candidates.empty else 0.5
high_thr = high_candidates["threshold"].min() if not high_candidates.empty else 0.8

print("Suggested thresholds:")
print("LOW threshold   (auto-approve max):", round(low_thr, 3))
print("MEDIUM threshold(soft check start):", round(med_thr, 3))
print("HIGH threshold  (hard check start):", round(high_thr, 3))

print("\nLOW row:")
display(thr_df[thr_df["threshold"] == low_thr])

print("\nMED row:")
display(thr_df[thr_df["threshold"] == med_thr])

print("\nHIGH row:")
display(thr_df[thr_df["threshold"] == high_thr])


Suggested thresholds:
LOW threshold   (auto-approve max): 1.0
MEDIUM threshold(soft check start): 0.26
HIGH threshold  (hard check start): 0.8

LOW row:


,threshold,precision,recall,FPR,TP,FP,FN,TN
100,1.0,0.0,0.0,0.0,0,0,30,2520



MED row:


,threshold,precision,recall,FPR,TP,FP,FN,TN
26,0.26,0.365854,0.5,0.010317,15,26,15,2494



HIGH row:


,threshold,precision,recall,FPR,TP,FP,FN,TN
80,0.8,1.0,0.4,0.0,12,0,18,2520


In [30]:
df_proc.columns

Index(['client_id', 'transdate', 'transdatetime', 'amount', 'transaction_id',
       'destination_id', 'is_fraud', 'trans_date', 'os_ver_cnt_30d',
       'phone_model_cnt_30d', 'phone_model_last', 'os_version_last',
       'login_sessions_7d', 'login_sessions_30d', 'logins_per_day_7d',
       'logins_per_day_30d', 'login_freq_change_7d_vs_30d',
       'logins_7d_share_of_30d', 'avg_session_interval_30d',
       'std_session_interval_30d', 'var_session_interval_30d',
       'ewm_session_interval_7d', 'burstiness_sessions',
       'fano_factor_sessions', 'zscore_interval_7d_vs_30d',
       'amount_was_missing', 'os_ver_cnt_30d_was_missing',
       'phone_model_cnt_30d_was_missing', 'login_sessions_7d_was_missing',
       'login_sessions_30d_was_missing', 'logins_per_day_7d_was_missing',
       'logins_per_day_30d_was_missing',
       'login_freq_change_7d_vs_30d_was_missing',
       'logins_7d_share_of_30d_was_missing',
       'avg_session_interval_30d_was_missing',
       'std_session_i

In [31]:
df_proc["os_version_last"].unique()

array([29,  4,  5, 46,  1, 39,  2,  0, 17, 28, 43,  3, 20, 31, 38, 45, 40,
       36, 41, 32, 35, 24, 12, 22, 52, 27, 19, 42, 33, 23,  6, 48, 16, 15,
       11, 51, 37, 34, 26, 44, 21, 47,  7, 13, 30, 53, 18, 14, 49, 50,  8,
       25, 54, 10,  9])